# 청구입금정보 EDA

In [1]:
import sys
sys.path.append('../utils')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn import set_config
from preprocessing import (
    DateElapsedTransformer, NumericFeaturePreprocessor, ObjectFeaturePreprocessor, NumericTypeOptimizer,
)

pre_optimizer = NumericTypeOptimizer(mode='pre')
date_transformer = DateElapsedTransformer()
numeric_transformer = NumericFeaturePreprocessor()
object_transformer = ObjectFeaturePreprocessor()
post_optimizer = NumericTypeOptimizer(mode='post')

preprocessing_pipeline = Pipeline([
    ('pre_optimizer', pre_optimizer),
    ('object', object_transformer),
    ('date', date_transformer),
    ('numeric', numeric_transformer),
    ('post_optimizer', post_optimizer),
])

# Set display configuration to visualize the pipeline
set_config(display='diagram')

# Display the pipeline
preprocessing_pipeline

Pipeline(steps=[('pre_optimizer', NumericTypeOptimizer(mode='pre')),
                ('object',
                 ObjectFeaturePreprocessor(exclude_columns=['ID', 'Segment'])),
                ('date',
                 DateElapsedTransformer(base_date=Timestamp('2019-01-01 00:00:00'),
                                        exclude_columns=['ID', 'Segment',
                                                         '기준년월'])),
                ('numeric',
                 NumericFeaturePreprocessor(exclude_columns=['기준년월'])),
                ('post_optimizer', NumericTypeOptimizer())])

In [3]:
from sklearn.preprocessing import LabelEncoder
from data_loader import (
    load_data,
    load_segment
)

bill_df, bill_test_df = load_data("4.청구입금정보")

X = bill_df.drop(columns=['ID', '기준년월'])
X = preprocessing_pipeline.fit_transform(X)
X_test = preprocessing_pipeline.transform(bill_test_df.drop(columns=['ID', '기준년월']))

y = load_segment()
le = LabelEncoder()
y_encoded = le.fit_transform(y)


✅ File: ../../dataset/train/4.청구입금정보\201807_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201808_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201809_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201810_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201811_train_청구정보.parquet Completed!
✅ File: ../../dataset/train/4.청구입금정보\201812_train_청구정보.parquet Completed!
🔹 Shape : (2400000, 46)

✅ File: ../../dataset/test/4.청구입금정보\201807_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201808_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201809_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201810_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201811_test_청구정보.parquet Completed!
✅ File: ../../dataset/test/4.청구입금정보\201812_test_청구정보.parquet Completed!
🔹 Shape : (600000, 46)
Numeric Type Optimizer Transforming...
🧠 [mode=pre] 메모리 최적화: 1926.41 MB → 14

## Feature Selection

In [4]:
from feature import FeatureSelector

feature_engineering_pipeline = Pipeline([
    ('feature_selector', FeatureSelector()),
    ('optimize', post_optimizer),
])

X = feature_engineering_pipeline.fit_transform(X, y_encoded)
X_test = feature_engineering_pipeline.transform(X_test)

🎯 Fitting RandomForest for Feature Selection...
✅ Selected 9 features out of 56
📊 Top Selected Features by Importance:
1) 	청구금액_B0 (0.1583)
2) 	청구금액_R3M (0.2118)
3) 	청구금액_R6M (0.2212)
4) 	포인트_적립포인트_R12M (0.0273)
5) 	포인트_이용포인트_R12M (0.0278)
6) 	혜택수혜금액_R3M (0.0537)
7) 	포인트_마일리지_환산_B0M (0.0191)
8) 	혜택수혜금액 (0.0493)
9) 	할인건수_R3M_1회 이상 (0.0254)


c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Numeric Type Optimizer Transforming...
🧠 [mode=post] 메모리 최적화: 164.80 MB → 61.80 MB (62.5% 감소)
Numeric Type Optimizer Transforming...
🧠 [mode=post] 메모리 최적화: 41.20 MB → 15.45 MB (62.5% 감소)


c:\Users\Lee JinSeoung\OneDrive\Desktop\4학년 1학기\고급프로그래밍\과제\Credit-Card-Segment-Classfication\.venv\lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [5]:
import pandas as pd

train_df = pd.concat([bill_df[['ID', '기준년월']], X], axis=1)
test_df = pd.concat([bill_test_df[['ID', '기준년월']], X_test], axis=1)

train_df.to_parquet('../../dataset/train/bill.parquet', index=False)
test_df.to_parquet('../../dataset/test/bill.parquet', index=False)